In [1]:
# Package importation
import numpy as np
import cv2
import pickle
# import PIL.ExifTags
# import PIL.Image

In [2]:
# CHECKERBOARD = (8,6)
CHECKERBOARD =(7,7)

# Termination criteria
criteria =(cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
criteria_stereo= (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

# Prepare object points
objp = np.zeros((1, CHECKERBOARD[0]*CHECKERBOARD[1], 3), np.float32)
objp[0,:,:2] = np.mgrid[0:CHECKERBOARD[0], 0:CHECKERBOARD[1]].T.reshape(-1, 2)

# Prepare object points
# objp = np.zeros((9*6,3), np.float32)
# objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)
# Arrays to store object points and image points from all images
objpoints= []   # 3d points in real world space
imgpointsR= []   # 2d points in image plane
imgpointsL= []


In [3]:
#The code works,,,,any error thrown is due to the images

# Start calibration from the camera
print('Starting calibration for the 2 cameras... ')
# Call all saved images
total_imgs = 36
for i in range(0,total_imgs):   # Put the amount of pictures you have taken for the calibration inbetween range(0,?) wenn starting from the image number 0
    t= str(i)
    ChessImaR= cv2.imread('calib_imgs/camera_right'+t+'.png',0)    # Right side
    ChessImaL= cv2.imread('calib_imgs/camera_left'+t+'.png',0)    # Left side
    retR, cornersR = cv2.findChessboardCorners(ChessImaR,CHECKERBOARD,cv2.CALIB_CB_ADAPTIVE_THRESH+cv2.CALIB_CB_FAST_CHECK+cv2.CALIB_CB_NORMALIZE_IMAGE)  # Define the number of chees corners we are looking for
    retL, cornersL = cv2.findChessboardCorners(ChessImaL,CHECKERBOARD,cv2.CALIB_CB_ADAPTIVE_THRESH+cv2.CALIB_CB_FAST_CHECK+cv2.CALIB_CB_NORMALIZE_IMAGE)  # Left side
    print("level_",t,retR,retL)
    if (True == retR) & (True == retL):
        objpoints.append(objp)
        cv2.cornerSubPix(ChessImaR,cornersR,(3,3),(-1,-1),criteria)
        cv2.cornerSubPix(ChessImaL,cornersL,(3,3),(-1,-1),criteria)
        imgpointsR.append(cornersR)
        imgpointsL.append(cornersL)

Starting calibration for the 2 cameras... 
level_ 0 True True
level_ 1 True False
level_ 2 True True
level_ 3 True False
level_ 4 True False
level_ 5 True False
level_ 6 True True
level_ 7 True False
level_ 8 False False
level_ 9 True True
level_ 10 True True
level_ 11 True True
level_ 12 True True
level_ 13 False False
level_ 14 True True
level_ 15 False False
level_ 16 False False
level_ 17 False False
level_ 18 False False
level_ 19 False False
level_ 20 True True
level_ 21 True True
level_ 22 True True
level_ 23 True True
level_ 24 True True
level_ 25 True True
level_ 26 False True
level_ 27 False False
level_ 28 False True
level_ 29 False True
level_ 30 False True
level_ 31 False True
level_ 32 False True
level_ 33 False True
level_ 34 True True
level_ 35 False True


In [ ]:
# print(imgpointsR)

In [4]:
# print(imgpointsL,imgpointsR)
# Determine the new values for different parameters
#   Right Side
retR, mtxR, distR, rvecsR, tvecsR = cv2.calibrateCamera(objpoints,  imgpointsR,  ChessImaR.shape[::-1],None,None)

hR,wR= ChessImaR.shape[:2]
OmtxR, roiR= cv2.getOptimalNewCameraMatrix(mtxR,distR,(wR,hR),1,(wR,hR))

#   Left Side
retL, mtxL, distL, rvecsL, tvecsL = cv2.calibrateCamera(objpoints,imgpointsL,ChessImaL.shape[::-1],None,None)

hL,wL= ChessImaL.shape[:2]
OmtxL, roiL= cv2.getOptimalNewCameraMatrix(mtxL,distL,(wL,hL),1,(wL,hL))

print('Cameras Ready to use')
print("Mls",mtxL)
print()
print("Mrs",mtxR)

Cameras Ready to use
Mls [[572.17647393   0.         478.96299565]
 [  0.         572.18235571 359.12679712]
 [  0.           0.           1.        ]]

Mrs [[571.47628153   0.         479.37913581]
 [  0.         571.45085247 359.15989811]
 [  0.           0.           1.        ]]


In [5]:
#********************************************
#***** Calibrate the Cameras for Stereo *****
#********************************************

# StereoCalibrate function
flags = 0
flags |= cv2.CALIB_FIX_INTRINSIC
#flags |= cv2.CALIB_FIX_PRINCIPAL_POINT
#flags |= cv2.CALIB_USE_INTRINSIC_GUESS
#flags |= cv2.CALIB_FIX_FOCAL_LENGTH
#flags |= cv2.CALIB_FIX_ASPECT_RATIO
#flags |= cv2.CALIB_ZERO_TANGENT_DIST
#flags |= cv2.CALIB_RATIONAL_MODEL
#flags |= cv2.CALIB_SAME_FOCAL_LENGTH
#flags |= cv2.CALIB_FIX_K3
#flags |= cv2.CALIB_FIX_K4
#flags |= cv2.CALIB_FIX_K5
retS, MLS, dLS, MRS, dRS, R, T, E, F= cv2.stereoCalibrate(objpoints,
                                                          imgpointsL,
                                                          imgpointsR,
                                                          mtxL,
                                                          distL,
                                                          mtxR,
                                                          distR,
                                                          ChessImaR.shape[::-1],
                                                          criteria_stereo,
                                                          flags)


In [6]:
# StereoRectify function
rectify_scale= 0 # if 0 image croped, if 1 image nor croped
RL, RR, PL, PR, Q, roiL, roiR= cv2.stereoRectify(MLS, dLS, MRS, dRS,
                                                 ChessImaR.shape[::-1], R, T,
                                                 rectify_scale,(0,0))  # last paramater is alpha, if 0= croped, if 1= not croped
# initUndistortRectifyMap function
Left_Stereo_Map= cv2.initUndistortRectifyMap(MLS, dLS, RL, PL,
                                             ChessImaR.shape[::-1], cv2.CV_16SC2)   # cv2.CV_16SC2 this format enables us the programme to work faster
Right_Stereo_Map= cv2.initUndistortRectifyMap(MRS, dRS, RR, PR,
                                              ChessImaR.shape[::-1], cv2.CV_16SC2)
print(Q)

[[   1.            0.            0.         -477.64002609]
 [   0.            1.            0.         -439.02316284]
 [   0.            0.            0.          571.82637773]
 [   0.            0.            1.65088716   -0.        ]]


In [9]:
val_left = "left_map"
# print(Left_Stereo_Map)
left_map = open(val_left, "wb")#Open the file
pickle.dump(Left_Stereo_Map, left_map, protocol=2)#Dump the dictionary bok, the first parameter into the file object w.
left_map.close()

val_right = "right_map"
# print(Left_Stereo_Map)
right_map = open(val_right, "wb")#Open the file
pickle.dump(Right_Stereo_Map, right_map, protocol=2) #protocol= 2 because I use a python2 to open, if not, remove
right_map.close()

In [7]:
# import pickle#import the pickle module,protocol=2
# w=open(val, 'rb')#Open the file
# bok=pickle.load(w)#Assign the recreated object to bok
# # print(bok)

left_map = "left_map"
right_map = "right_map"
w_left = open(left_map, 'rb')  # Open the file
w_right = open(right_map, 'rb')  # Open the file
ap = pickle.load(w_left)  # Assign the recreated object to bok
print(ap)

(array([[[-15,  15],
        [-14,  15],
        [-13,  15],
        ...,
        [968,  11],
        [969,  11],
        [970,  11]],

       [[-15,  16],
        [-14,  16],
        [-13,  16],
        ...,
        [968,  12],
        [969,  12],
        [970,  12]],

       [[-15,  17],
        [-14,  17],
        [-13,  17],
        ...,
        [968,  13],
        [969,  13],
        [970,  13]],

       ...,

       [[ 13, 731],
        [ 14, 731],
        [ 15, 731],
        ...,
        [946, 727],
        [947, 727],
        [948, 727]],

       [[ 13, 732],
        [ 14, 732],
        [ 15, 732],
        ...,
        [946, 728],
        [947, 728],
        [948, 728]],

       [[ 13, 733],
        [ 14, 733],
        [ 15, 733],
        ...,
        [946, 729],
        [947, 729],
        [948, 729]]], dtype=int16), array([[686, 720, 721, ..., 856, 857, 827],
       [752, 785, 787, ..., 919, 920, 889],
       [818, 851, 853, ..., 981, 983, 952],
       ...,
       [564, 564, 

In [ ]:
# https://medium.com/@omar.ps16/stereo-3d-reconstruction-with-opencv-using-an-iphone-camera-part-ii-77754b58bfe0
#Get exif data in order to get focal length. 
exif_img = PIL.Image.open(calibration_paths[0])exif_data = {
 PIL.ExifTags.TAGS[k]:v
 for k, v in exif_img._getexif().items()
 if k in PIL.ExifTags.TAGS}#Get focal length in tuple form
focal_length_exif = exif_data['FocalLength']#Get focal length in decimal form
focal_length = focal_length_exif[0]/focal_length_exif[1]
print(focal_length)
# np.save("./camera_params/FocalLength", focal_length)